# 2.2.1 Prompting Techniques & Patterns

## Playground Notebook

This notebook covers the **core prompting strategies** that turn a generic LLM into a precise, reliable tool.

| Technique | Core Idea |
|-----------|----------|
| **Zero-shot** | Ask directly — no examples needed |
| **One-shot / Few-shot** | Teach by showing 1 or more examples |
| **Role-based** | Assign a persona to shape tone, depth, and perspective |
| **Structured Output** | Force responses into JSON, XML, Markdown, etc. |
| **Prompt Templates** | Reusable patterns with placeholders for different inputs |

---

In [25]:
import json
import time
from IPython.display import display, Markdown, HTML
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# ============================================================
#  CONFIGURATION - Change the model name here if needed
# ============================================================
MODEL = "qwen2.5:1.5b"

llm = ChatOllama(model=MODEL)

# ============================================================
#  HELPER FUNCTIONS
# ============================================================

def build_messages(message_dicts):
    """Convert a list of role/content dicts into LangChain message objects."""
    type_map = {
        "system": SystemMessage,
        "user": HumanMessage,
        "assistant": AIMessage,
    }
    return [type_map[m["role"]](content=m["content"]) for m in message_dicts]


def chat(messages, **kwargs):
    """Send messages to the model and return the response text."""
    _llm = ChatOllama(model=MODEL, **kwargs) if kwargs else llm
    lc_messages = build_messages(messages)
    start = time.time()
    response = _llm.invoke(lc_messages)
    elapsed = time.time() - start
    content = response.content
    display(Markdown(content))
    print(f"\n⏱️ {elapsed:.2f}s | {len(content)} chars")
    return content


def show_messages(messages):
    """Pretty-print the message list being sent to the model."""
    colors = {"system": "#e74c3c", "user": "#3498db", "assistant": "#2ecc71"}
    html = ""
    for msg in messages:
        role = msg["role"]
        color = colors.get(role, "#888")
        content_preview = msg["content"][:300] + ("..." if len(msg["content"]) > 300 else "")
        html += (
            f'<div style="margin:6px 0;padding:8px 12px;border-left:4px solid {color};'
            f'background:#1e1e1e;border-radius:4px;">'
            f'<strong style="color:{color};text-transform:uppercase;">{role}</strong>'
            f'<br><span style="color:#ccc;white-space:pre-wrap;">{content_preview}</span></div>'
        )
    display(HTML(html))


print(f"✅ Using model: {MODEL}")

✅ Using model: qwen2.5:1.5b


---

## 1. Zero-Shot Prompting — No Examples Needed

In **zero-shot** prompting, you give the model a task with **no examples** at all. You rely entirely on the model's pre-trained knowledge and clear instructions.

```
Zero-shot = Instruction only, no demonstrations
```

This works well when:
- The task is straightforward (classification, summarization, translation)
- The model already understands the format you want
- You want quick results without crafting examples

### Experiment 1A: Basic Zero-Shot Tasks

In [26]:
zero_shot_tasks = [
    {
        "label": "Sentiment Classification",
        "prompt": "Classify the sentiment of this review as Positive, Negative, or Neutral. Reply with just the label.\n\nReview: \"The battery life is incredible but the screen is too dim outdoors.\""
    },
    {
        "label": "Translation",
        "prompt": "Translate the following English sentence to Spanish:\n\n\"The weather is beautiful today and I want to go for a walk.\""
    },
    {
        "label": "Summarization",
        "prompt": "Summarize the following paragraph in one sentence:\n\n\"Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed. It focuses on developing computer programs that can access data and use it to learn for themselves. The process begins with observations or data, such as examples, direct experience, or instruction, in order to look for patterns in data and make better decisions in the future.\""
    }
]

for task in zero_shot_tasks:
    print(f"\n{'=' * 60}")
    print(f"  ZERO-SHOT: {task['label']}")
    print(f"{'=' * 60}")
    messages = [{"role": "user", "content": task["prompt"]}]
    show_messages(messages)
    _ = chat(messages, temperature=0.0)


  ZERO-SHOT: Sentiment Classification


Negative


⏱️ 0.28s | 8 chars

  ZERO-SHOT: Translation


La traducción al español sería: "El clima es hermoso hoy y quiero hacer una caminata."


⏱️ 0.35s | 86 chars

  ZERO-SHOT: Summarization


Machine learning is a branch of artificial intelligence that allows systems to automatically improve through experience without explicit programming.


⏱️ 0.35s | 149 chars


### Experiment 1B: When Zero-Shot Struggles

Zero-shot can fail on **ambiguous, nuanced, or domain-specific** tasks where the model doesn't know the exact format you expect.

In [27]:
# A task where zero-shot may give inconsistent formatting
prompt = """Extract the product name, price, and currency from this text:

"I just bought the Sony WH-1000XM5 headphones for €349.99 at the airport."
"""

print("ZERO-SHOT EXTRACTION (no format guidance)")
print("=" * 60)
messages = [{"role": "user", "content": prompt}]
show_messages(messages)
result_zs = chat(messages, temperature=0.0)

print("\n💡 Notice: The model returns an answer, but the format is unpredictable.")
print("   We'll fix this with few-shot examples later!")

ZERO-SHOT EXTRACTION (no format guidance)


Product Name: Sony WH-1000XM5 headphones
Price: €349.99
Currency: Euro (€)


⏱️ 0.41s | 74 chars

💡 Notice: The model returns an answer, but the format is unpredictable.
   We'll fix this with few-shot examples later!


---

## 2. One-Shot & Few-Shot Prompting — Teaching by Example

Instead of just *telling* the model what to do, you *show* it with examples:

```
Zero-shot:  "Classify this sentiment"            → 0 examples
One-shot:   "Here's one example, now do this"     → 1 example
Few-shot:   "Here are 3 examples, now do this"    → 2-5 examples
```

Examples are provided as **user → assistant** message pairs in the conversation history.

### Experiment 2A: Zero-Shot vs. One-Shot vs. Few-Shot

In [28]:
# ---- ZERO-SHOT ----
print("=" * 60)
print("  ZERO-SHOT (0 examples)")
print("=" * 60)

messages_zero = [
    {"role": "system", "content": "Classify the emotion in each sentence. Reply with exactly one word: Happy, Sad, Angry, or Fearful."},
    {"role": "user", "content": "I can't believe they cancelled the concert after I waited all year."}
]
show_messages(messages_zero)
_ = chat(messages_zero, temperature=0.0)

  ZERO-SHOT (0 examples)


Sad


⏱️ 0.14s | 3 chars


In [29]:
# ---- ONE-SHOT ----
print("=" * 60)
print("  ONE-SHOT (1 example)")
print("=" * 60)

messages_one = [
    {"role": "system", "content": "Classify the emotion in each sentence. Reply with exactly one word: Happy, Sad, Angry, or Fearful."},
    {"role": "user", "content": "I just got promoted at work!"},
    {"role": "assistant", "content": "Happy"},
    {"role": "user", "content": "I can't believe they cancelled the concert after I waited all year."}
]
show_messages(messages_one)
_ = chat(messages_one, temperature=0.0)

  ONE-SHOT (1 example)


Sad


⏱️ 0.14s | 3 chars


In [30]:
# ---- FEW-SHOT ----
print("=" * 60)
print("  FEW-SHOT (3 examples)")
print("=" * 60)

messages_few = [
    {"role": "system", "content": "Classify the emotion in each sentence. Reply with exactly one word: Happy, Sad, Angry, or Fearful."},
    # Example 1
    {"role": "user", "content": "I just got promoted at work!"},
    {"role": "assistant", "content": "Happy"},
    # Example 2
    {"role": "user", "content": "My dog passed away last night."},
    {"role": "assistant", "content": "Sad"},
    # Example 3
    {"role": "user", "content": "There's a strange noise coming from the basement."},
    {"role": "assistant", "content": "Fearful"},
    # Actual task
    {"role": "user", "content": "I can't believe they cancelled the concert after I waited all year."}
]
show_messages(messages_few)
_ = chat(messages_few, temperature=0.0)

  FEW-SHOT (3 examples)


Angry


⏱️ 0.15s | 5 chars


### Experiment 2B: Few-Shot Fixes Formatting Problems

Remember the extraction task that had unpredictable formatting in zero-shot? Let's fix it with examples.

In [31]:
messages = [
    {"role": "system", "content": "Extract product name, price, and currency from the text. Use the exact format shown in the examples."},
    # Example 1
    {"role": "user", "content": "I purchased the Apple MacBook Air for $1,299 from the online store."},
    {"role": "assistant", "content": "Product: Apple MacBook Air\nPrice: 1299\nCurrency: USD"},
    # Example 2
    {"role": "user", "content": "The Samsung Galaxy S24 was on sale for £799 at Heathrow."},
    {"role": "assistant", "content": "Product: Samsung Galaxy S24\nPrice: 799\nCurrency: GBP"},
    # Actual task
    {"role": "user", "content": "I just bought the Sony WH-1000XM5 headphones for €349.99 at the airport."}
]

print("FEW-SHOT EXTRACTION (2 examples → consistent format)")
print("=" * 60)
show_messages(messages)
_ = chat(messages, temperature=0.0)

print("\n💡 Compare this to the zero-shot result above — the format is now consistent!")

FEW-SHOT EXTRACTION (2 examples → consistent format)


Product: Sony WH-1000XM5
Price: 349.99
Currency: EUR


⏱️ 0.45s | 52 chars

💡 Compare this to the zero-shot result above — the format is now consistent!


### Experiment 2C: Few-Shot for Custom Classification

Few-shot is especially powerful for **custom categories** that don't exist in standard training data.

In [32]:
# Custom support ticket classification — categories specific to a fictional company
messages = [
    {"role": "system", "content": "Classify each support ticket into one category: BILLING, BUG, FEATURE_REQUEST, or ACCOUNT_ACCESS. Reply with the category only."},
    # Examples teach the model our specific category definitions
    {"role": "user", "content": "I was charged twice for my subscription this month."},
    {"role": "assistant", "content": "BILLING"},
    {"role": "user", "content": "The export button crashes the app on Safari."},
    {"role": "assistant", "content": "BUG"},
    {"role": "user", "content": "It would be great if you added dark mode."},
    {"role": "assistant", "content": "FEATURE_REQUEST"},
    {"role": "user", "content": "I forgot my password and the reset email never arrived."},
    {"role": "assistant", "content": "ACCOUNT_ACCESS"},
    # New tickets to classify
    {"role": "user", "content": "Can you add the ability to schedule reports weekly?"}
]

print("FEW-SHOT CUSTOM CLASSIFICATION")
print("=" * 60)
show_messages(messages)
_ = chat(messages, temperature=0.0)

# Test with another ticket
messages[-1] = {"role": "user", "content": "My invoice shows the wrong company name."}
print("\n--- Next ticket ---")
_ = chat(messages, temperature=0.0)

messages[-1] = {"role": "user", "content": "The page goes blank when I click on my profile settings."}
print("\n--- Next ticket ---")
_ = chat(messages, temperature=0.0)

FEW-SHOT CUSTOM CLASSIFICATION


FEATURE_REQUEST


⏱️ 0.21s | 15 chars

--- Next ticket ---


BILLING


⏱️ 0.13s | 7 chars

--- Next ticket ---


BUG


⏱️ 0.13s | 3 chars


---

## 3. Role-Based Prompting — Assigning Personas

By assigning a **role** or **persona** via the system prompt, you control:
- **Expertise level** — beginner-friendly vs. expert-level depth
- **Tone** — formal, casual, enthusiastic, clinical
- **Perspective** — what the model prioritizes and how it frames answers
- **Constraints** — what the model should and shouldn't discuss

### Experiment 3A: Same Question, Different Expert Roles

In [33]:
question = "Why is my Python program running slowly?"

roles = [
    {
        "label": "Junior Developer Mentor",
        "system": "You are a patient senior developer mentoring a junior. Use simple language, avoid jargon, and give practical first steps. Keep it to 3-4 bullet points."
    },
    {
        "label": "Performance Engineer",
        "system": "You are a performance engineer specializing in Python optimization. Be technical and specific — mention profiling tools, algorithmic complexity, and memory management. Keep it to 3-4 bullet points."
    },
    {
        "label": "Sarcastic Code Reviewer",
        "system": "You are a brutally honest but funny code reviewer. Point out common mistakes with dry humor. Keep it to 3-4 bullet points."
    }
]

for role in roles:
    print(f"\n{'=' * 60}")
    print(f"  ROLE: {role['label']}")
    print(f"{'=' * 60}")
    messages = [
        {"role": "system", "content": role["system"]},
        {"role": "user", "content": question}
    ]
    show_messages(messages)
    _ = chat(messages, temperature=0.7)


  ROLE: Junior Developer Mentor


Your Python program might be slow for several reasons:

1. **Code Complexity**: If your code contains a lot of complex operations or if you are doing repetitive tasks, this can slow down the execution time significantly.

2. **Database Queries**: Slow database queries can cause your application to run slower. Make sure that your SQL queries are optimized and not returning unnecessary data.

3. **File I/O Operations**: Reading from or writing to files can take a long time if they aren't done efficiently, especially with large datasets.

4. **Network Latency**: If you're dealing with external APIs or network requests frequently, these operations can be very slow if the underlying infrastructure isn't optimized for frequent access.

5. **Concurrency Issues**: If your program is trying to do multiple tasks at once without proper synchronization (like race conditions), it can lead to significant slowdowns.

### Practical Steps:

1. **Profile Your Code**: Use a profiling tool like `cProfile` or `pyprof2calltree` from the `pstats` module in Python, which helps you identify the parts of your program that take most time and resources.

2. **Optimize Database Queries**: Analyze your database queries using tools like `sqlite3`, `psycopg2` (for PostgreSQL), etc., to ensure they are as efficient as possible.

3. **Cache File Reads/Writes**: If reading/writing files is a bottleneck, consider caching the data or making it persistent if you can.

4. **Optimize Network Calls**: For network operations, check if your internet connection is slow and try improving your network configuration or using faster alternatives like cloud services that provide high-speed access.

5. **Avoid Threading Conflicts**: Use synchronization primitives (locks, semaphores) to ensure that multiple threads are not accessing shared resources improperly.

By addressing these points, you should be able to identify the root cause of the slow performance and take effective measures to improve it.


⏱️ 4.50s | 1984 chars

  ROLE: Performance Engineer


1. **Profiling Tools Utilization**: Employing profiling tools like `cProfile`, `py-spy`, or the built-in `cProfile` module can help identify bottlenecks in your code by analyzing function call times and memory usage. This tool helps pinpoint where the majority of the time is being spent, which is crucial for optimizing performance.

2. **Algorithmic Complexity Analysis**: Understanding the algorithmic complexity of your functions can reveal why certain operations are slowing down. For instance, a quadratic operation (O(n^2)) compared to an optimal linear one (O(n)) would significantly impact runtime efficiency. Profiling helps in determining if the chosen algorithms are efficient or need optimization.

3. **Memory Management**: Efficient memory management is key to optimizing Python performance, especially concerning garbage collection and handling large data structures. Techniques such as using `with` statements for context managers can help manage resources more effectively, reducing unnecessary object creation and thus improving speed. Additionally, understanding the impact of global variables on memory usage during function calls helps in minimizing wasted memory.

4. **Code Optimization**: Implementing optimizations directly in your code can often yield performance improvements without needing to resort to profiling or algorithmic changes. This could include replacing loops with vectorized operations (e.g., using NumPy arrays) for faster data handling, caching results of expensive calculations, and avoiding deep copying when passing large objects as function parameters.


⏱️ 3.22s | 1601 chars

  ROLE: Sarcastic Code Reviewer


1. **The Loop Isn't Optimized**: "I’m not going down that rabbit hole of profiling, but trust me, your loop might be bloating up like a water balloon and slowing everything to a crawl."

2. **Database Queries Too Many**: "Your database is feeling the effects of too many queries, just like when you try to clean a room with a sledgehammer instead of a vacuum cleaner. It’s not only messy but also super slow!"

3. **Global Variables Are Killing Your Performance**: "Oh no, your program is using global variables as if they were magic potion ingredients. Each function call shuffles them around, and before you know it, your performance has shrunk to the size of a grain of rice."


⏱️ 1.85s | 679 chars


### Experiment 3B: Role-Based for Audience Adaptation

The same topic explained to **different audiences** using role prompts.

In [34]:
topic = "Explain what an API is."

audiences = [
    {
        "label": "For a 10-year-old",
        "system": "You are a fun science teacher at a primary school. Explain concepts using everyday analogies a 10-year-old would understand. Keep it to 2-3 sentences."
    },
    {
        "label": "For a business executive",
        "system": "You are a technology consultant presenting to a non-technical CEO. Focus on business value, not implementation details. Keep it to 2-3 sentences."
    },
    {
        "label": "For a software engineer",
        "system": "You are a senior backend architect. Be precise and technical. Mention protocols, patterns, and relevant standards. Keep it to 2-3 sentences."
    }
]

for aud in audiences:
    print(f"\n{'=' * 60}")
    print(f"  AUDIENCE: {aud['label']}")
    print(f"{'=' * 60}")
    messages = [
        {"role": "system", "content": aud["system"]},
        {"role": "user", "content": topic}
    ]
    show_messages(messages)
    _ = chat(messages, temperature=0.5)


  AUDIENCE: For a 10-year-old


Imagine you have a magical key that can unlock any door in your house, but the key doesn't come with instructions on how to use it. That's kind of like how APIs work. An API (Application Programming Interface) is like a secret code or set of rules for talking to other programs or websites, but just as you don’t need to know all the steps to open a door, developers don’t always need to understand every detail about an API. They can use it without knowing everything in advance. It’s just another way that technology helps people do things more easily and efficiently!


⏱️ 1.36s | 570 chars

  AUDIENCE: For a business executive


An API (Application Programming Interface) is like a set of rules and protocols that different software applications can use to communicate with each other. It's essentially a way for one application or service to request specific actions from another, much like how you might ask your car to start by pressing the button on the dashboard instead of having to know every intricate detail about its engine. This makes it easier for developers and businesses to integrate different systems seamlessly without needing deep knowledge of each other's inner workings.


⏱️ 1.25s | 561 chars

  AUDIENCE: For a software engineer


An Application Programming Interface (API) is a set of rules and protocols that allow different software applications to communicate with each other. It defines the functions and operations available for use by other programs, enabling them to interact seamlessly. APIs can be used to integrate systems, services, or data sources, making it easier to build complex applications without needing deep knowledge of underlying technologies.


⏱️ 0.86s | 436 chars


### Experiment 3C: Role + Few-Shot Combined

The most effective prompts often **combine techniques**. Here we pair a role with few-shot examples.

In [35]:
messages = [
    {
        "role": "system",
        "content": (
            "You are a medical triage assistant at a hospital reception. "
            "Based on the patient's symptoms, assign a triage level: "
            "GREEN (non-urgent), YELLOW (semi-urgent), or RED (emergency). "
            "Format: LEVEL — brief reason (one sentence)."
        )
    },
    # Few-shot examples
    {"role": "user", "content": "Patient has a mild headache that started this morning."},
    {"role": "assistant", "content": "GREEN — Mild headache with no other symptoms suggests a non-urgent condition."},
    {"role": "user", "content": "Patient reports chest pain radiating to the left arm with shortness of breath."},
    {"role": "assistant", "content": "RED — Chest pain with arm radiation and dyspnea indicates possible myocardial infarction."},
    {"role": "user", "content": "Patient has a deep cut on the forearm, bleeding is controlled with pressure."},
    {"role": "assistant", "content": "YELLOW — Deep laceration needs medical attention but bleeding is currently controlled."},
    # Actual cases
    {"role": "user", "content": "Patient has a high fever of 39.5°C, stiff neck, and sensitivity to light."}
]

print("ROLE + FEW-SHOT: Medical Triage Assistant")
print("=" * 60)
show_messages(messages)
_ = chat(messages, temperature=0.0)

# Try another case
messages[-1] = {"role": "user", "content": "Patient has a runny nose and occasional sneezing for 2 days."}
print("\n--- Next patient ---")
_ = chat(messages, temperature=0.0)

ROLE + FEW-SHOT: Medical Triage Assistant


RED — High fever with neurological symptoms suggests meningitis, an emergency condition.


⏱️ 0.43s | 88 chars

--- Next patient ---


GREEN — Runny nose and occasional sneezing suggest common cold, non-urgent.


⏱️ 0.30s | 75 chars


---

## 4. Structured Output Generation

Getting the model to respond in a **specific format** (JSON, XML, Markdown) is critical for building reliable pipelines. Three strategies, in order of reliability:

1. **Zero-shot with format instructions** — tell the model what format you want
2. **Few-shot with format examples** — show the exact format you expect
3. **System prompt + few-shot** — combine role constraints with examples

### Experiment 4A: JSON Output — Zero-Shot vs. Few-Shot

In [36]:
input_text = "Marie Curie was born in Warsaw in 1867. She won two Nobel Prizes — one in Physics (1903) and one in Chemistry (1911)."

# ---- Zero-shot JSON ----
print("=" * 60)
print("  ZERO-SHOT JSON")
print("=" * 60)

messages_zs = [
    {"role": "system", "content": "Extract information from the text and respond with valid JSON only. No extra text. Keys: name, birthplace, birth_year, achievements (list of objects with field and year)."},
    {"role": "user", "content": input_text}
]
show_messages(messages_zs)
result_zs = chat(messages_zs, temperature=0.0)

try:
    json.loads(result_zs)
    print("✅ Valid JSON")
except json.JSONDecodeError:
    print("❌ Invalid JSON")

  ZERO-SHOT JSON


{
  "name": "Marie Curie",
  "birthplace": "Warsaw",
  "birth_year": 1867,
  "achievements": [
    {
      "and_year": 1903
    },
    {
      "and_year": 1911
    }
  ]
}


⏱️ 0.83s | 171 chars
✅ Valid JSON


In [37]:
# ---- Few-shot JSON (more reliable) ----
print("=" * 60)
print("  FEW-SHOT JSON")
print("=" * 60)

messages_fs = [
    {"role": "system", "content": "Extract information from the text. Respond with valid JSON only."},
    # Example
    {"role": "user", "content": "Albert Einstein was born in Ulm in 1879. He won the Nobel Prize in Physics in 1921."},
    {"role": "assistant", "content": json.dumps({
        "name": "Albert Einstein",
        "birthplace": "Ulm",
        "birth_year": 1879,
        "achievements": [{"field": "Physics", "year": 1921}]
    }, indent=2)},
    # Actual task
    {"role": "user", "content": input_text}
]
show_messages(messages_fs)
result_fs = chat(messages_fs, temperature=0.0)

try:
    parsed = json.loads(result_fs)
    print("✅ Valid JSON")
    print(json.dumps(parsed, indent=2))
except json.JSONDecodeError:
    print("❌ Invalid JSON")

  FEW-SHOT JSON


{
  "name": "Marie Curie",
  "birthplace": "Warsaw",
  "birth_year": 1867,
  "achievements": [
    {
      "field": "Physics",
      "year": 1903
    },
    {
      "field": "Chemistry",
      "year": 1911
    }
  ]
}


⏱️ 1.08s | 217 chars
✅ Valid JSON
{
  "name": "Marie Curie",
  "birthplace": "Warsaw",
  "birth_year": 1867,
  "achievements": [
    {
      "field": "Physics",
      "year": 1903
    },
    {
      "field": "Chemistry",
      "year": 1911
    }
  ]
}


### Experiment 4B: XML Output

In [38]:
messages = [
    {"role": "system", "content": "Convert the book information into XML format. Respond with valid XML only."},
    # Example
    {"role": "user", "content": "The Great Gatsby by F. Scott Fitzgerald, published 1925, genre: Fiction"},
    {"role": "assistant", "content": """<book>
  <title>The Great Gatsby</title>
  <author>F. Scott Fitzgerald</author>
  <year>1925</year>
  <genre>Fiction</genre>
</book>"""},
    # Actual task
    {"role": "user", "content": "A Brief History of Time by Stephen Hawking, published 1988, genre: Science"}
]

print("FEW-SHOT XML GENERATION")
print("=" * 60)
show_messages(messages)
_ = chat(messages, temperature=0.0)

FEW-SHOT XML GENERATION


<book>
  <title>A Brief History of Time</title>
  <author>Stephen Hawking</author>
  <year>1988</year>
  <genre>Science</genre>
</book>


⏱️ 0.66s | 135 chars


### Experiment 4C: Markdown Table Output

In [39]:
messages = [
    {
        "role": "system",
        "content": "Convert the raw data into a well-formatted Markdown table. Include a header row. Respond with only the Markdown table."
    },
    # Example
    {"role": "user", "content": "Languages: Python - 1991 - Guido van Rossum; JavaScript - 1995 - Brendan Eich"},
    {"role": "assistant", "content": """| Language | Year | Creator |
|----------|------|---------|
| Python | 1991 | Guido van Rossum |
| JavaScript | 1995 | Brendan Eich |"""},
    # Actual task
    {"role": "user", "content": "Frameworks: React - 2013 - Meta; Angular - 2016 - Google; Vue.js - 2014 - Evan You; Svelte - 2016 - Rich Harris"}
]

print("FEW-SHOT MARKDOWN TABLE")
print("=" * 60)
show_messages(messages)
result = chat(messages, temperature=0.0)

print("\n📋 Rendered table:")
display(Markdown(result))

FEW-SHOT MARKDOWN TABLE


| Framework | Year | Company |
|-----------|------|----------|
| React | 2013 | Meta    |
| Angular | 2016 | Google   |
| Vue.js | 2014 | Evan You  |
| Svelte | 2016 | Rich Harris |


⏱️ 0.91s | 181 chars

📋 Rendered table:


| Framework | Year | Company |
|-----------|------|----------|
| React | 2013 | Meta    |
| Angular | 2016 | Google   |
| Vue.js | 2014 | Evan You  |
| Svelte | 2016 | Rich Harris |

### Experiment 4D: Multiple Formats from the Same Data

Let's extract the same information in **three different formats** to see how format instructions change the output.

In [40]:
raw_data = """Company: Acme Corp. Revenue: $5.2M. Employees: 120. Founded: 2018. Headquarters: Austin, TX."""

formats = [
    {
        "label": "JSON",
        "system": "Extract company data from the text. Respond with valid JSON only.",
        "example_in": "Company: FooBar Inc. Revenue: $1M. Employees: 30. Founded: 2020. Headquarters: NYC.",
        "example_out": json.dumps({"company": "FooBar Inc.", "revenue": "$1M", "employees": 30, "founded": 2020, "hq": "NYC"}, indent=2)
    },
    {
        "label": "XML",
        "system": "Extract company data from the text. Respond with valid XML only.",
        "example_in": "Company: FooBar Inc. Revenue: $1M. Employees: 30. Founded: 2020. Headquarters: NYC.",
        "example_out": "<company>\n  <name>FooBar Inc.</name>\n  <revenue>$1M</revenue>\n  <employees>30</employees>\n  <founded>2020</founded>\n  <hq>NYC</hq>\n</company>"
    },
    {
        "label": "Markdown Key-Value",
        "system": "Extract company data from the text. Respond as a Markdown bullet list with bold keys.",
        "example_in": "Company: FooBar Inc. Revenue: $1M. Employees: 30. Founded: 2020. Headquarters: NYC.",
        "example_out": "- **Company:** FooBar Inc.\n- **Revenue:** $1M\n- **Employees:** 30\n- **Founded:** 2020\n- **HQ:** NYC"
    }
]

for fmt in formats:
    print(f"\n{'=' * 60}")
    print(f"  FORMAT: {fmt['label']}")
    print(f"{'=' * 60}")
    messages = [
        {"role": "system", "content": fmt["system"]},
        {"role": "user", "content": fmt["example_in"]},
        {"role": "assistant", "content": fmt["example_out"]},
        {"role": "user", "content": raw_data}
    ]
    _ = chat(messages, temperature=0.0)


  FORMAT: JSON


{
  "company": "Acme Corp.",
  "revenue": "$5.2M",
  "employees": 120,
  "founded": 2018,
  "hq": "Austin, TX"
}


⏱️ 0.77s | 112 chars

  FORMAT: XML


<company>
  <name>Acme Corp.</name>
  <revenue>$5.2M</revenue>
  <employees>120</employees>
  <founded>2018</founded>
  <hq>Austin, TX</hq>
</company>


⏱️ 0.81s | 150 chars

  FORMAT: Markdown Key-Value


- **Company:** Acme Corp.
- **Revenue:** $5.2M
- **Employees:** 120
- **Founded:** 2018
- **HQ:** Austin, TX


⏱️ 0.67s | 108 chars


---

## 5. Prompt Templates & Reusability Patterns

Instead of writing a new prompt from scratch every time, build **reusable templates** with placeholders. This makes your prompts:
- **Consistent** across inputs
- **Testable** with different data
- **Maintainable** as a single source of truth

### Experiment 5A: Basic Template with String Formatting

In [41]:
# A reusable template for product descriptions
PRODUCT_TEMPLATE = """Write a compelling product description for an e-commerce listing.

Product: {product_name}
Category: {category}
Key Features: {features}
Target Audience: {audience}

Requirements:
- Write exactly 2-3 sentences
- Highlight the main benefit first
- Include a call to action"""

# Use the template with different products
products = [
    {
        "product_name": "AeroFit Pro Running Shoes",
        "category": "Athletic Footwear",
        "features": "Lightweight mesh, carbon-fiber plate, 30mm stack height",
        "audience": "Competitive marathon runners"
    },
    {
        "product_name": "ZenBrew Smart Kettle",
        "category": "Kitchen Appliances",
        "features": "Temperature presets, app control, keep-warm function",
        "audience": "Tea and coffee enthusiasts"
    }
]

for product in products:
    print(f"\n{'=' * 60}")
    print(f"  PRODUCT: {product['product_name']}")
    print(f"{'=' * 60}")
    prompt = PRODUCT_TEMPLATE.format(**product)
    messages = [{"role": "user", "content": prompt}]
    show_messages(messages)
    _ = chat(messages, temperature=0.7)


  PRODUCT: AeroFit Pro Running Shoes


Introducing the AeroFit Pro Running Shoes, designed for competitive marathon runners like you. Elevate your performance with lightweight mesh uppers that cut through the wind, and a carbon-fiber plate that adds a touch of aerodynamic edge. The 30mm stack height ensures every stride is optimized for speed and efficiency. Lace it on confidently knowing these shoes will take you from start to finish in style and strength. Run faster, longer, with confidence – get your AeroFit Pro Running Shoes today!


⏱️ 1.25s | 502 chars

  PRODUCT: ZenBrew Smart Kettle


Introducing ZenBrew Smart Kettle - your perfect companion for tea lovers! Crafted with temperature presets, it ensures every cup is perfectly brewed. The sleek design and intuitive app control make it easy to adjust settings on-the-go, while the keep-warm function keeps your favorite brews warm all day long. Whether you're brewing a single shot or a full pot of coffee, ZenBrew Smart Kettle elevates your morning rituals with ease and precision. Upgrade your brewing experience now!


⏱️ 1.18s | 484 chars


### Experiment 5B: Template with System + User Separation

A well-designed template separates the **behavior** (system) from the **data** (user).

In [42]:
# Reusable template as a function
def code_review_prompt(code_snippet, language="Python"):
    """Generate a code review using a standard template."""
    return [
        {
            "role": "system",
            "content": (
                f"You are a senior {language} code reviewer. "
                "For every review, respond with exactly this format:\n"
                "**Summary:** (one sentence)\n"
                "**Issues:** (numbered list or 'None')\n"
                "**Suggestions:** (numbered list or 'None')\n"
                "**Rating:** (1-5 stars)"
            )
        },
        {
            "role": "user",
            "content": f"Review this {language} code:\n\n```{language.lower()}\n{code_snippet}\n```"
        }
    ]


# Use the template for different code snippets
snippets = [
    {
        "label": "Python — Questionable function",
        "lang": "Python",
        "code": """def process(data):
    result = []
    for i in range(len(data)):
        if data[i] != None:
            result.append(data[i] * 2)
    return result"""
    },
    {
        "label": "JavaScript — Clean function",
        "lang": "JavaScript",
        "code": """const filterAdults = (people) => 
  people.filter(person => person.age >= 18)
        .map(({ name, age }) => ({ name, age }));"""
    }
]

for snippet in snippets:
    print(f"\n{'=' * 60}")
    print(f"  {snippet['label']}")
    print(f"{'=' * 60}")
    messages = code_review_prompt(snippet["code"], snippet["lang"])
    show_messages(messages)
    _ = chat(messages, temperature=0.3)


  Python — Questionable function


**Summary:** The provided function processes a list of elements, doubling each element that is not `None`, and returns the modified list.

**Issues:**
1. **Unused Parameter**: The parameter `data` is unused in the function.
2. **Type Safety**: Consider using type hints to specify the expected types for better code readability and static type checking.
3. **Return Type Consistency**: Ensure that the return type matches the expected output, which seems to be a list of doubled elements.

**Suggestions:**
1. Remove the unused parameter `data` from the function signature.
2. Add type hints to specify the expected types for better code readability and static type checking.
3. Return a list of doubled elements instead of modifying the original data structure directly.

```python
def process(data):
    result = []
    for i in range(len(data)):
        if data[i] is not None:
            result.append(data[i] * 2)
    return result

# Example usage:
data = [1, None, 3, None, 5]
result = process(data)
print(result)  # Output: [2, None, 6, None, 10]
```


⏱️ 2.85s | 1059 chars

  JavaScript — Clean function


**Summary:** The provided function filters an array of objects representing people by their age and then maps those that are adults to a new format.

**Issues:**
- **None**

**Suggestions:**
- None

**Rating:** 4 stars


⏱️ 0.68s | 218 chars


### Experiment 5C: Template Library — Multiple Reusable Patterns

In production, you maintain a **library of templates** for different tasks.

In [43]:
# A mini template library
TEMPLATES = {
    "summarize": {
        "system": "You are a concise summarizer. Summarize the text in exactly {length}. No extra commentary.",
        "user": "Summarize this:\n\n{text}"
    },
    "translate": {
        "system": "You are a professional translator. Translate accurately while preserving tone and meaning.",
        "user": "Translate the following from {source_lang} to {target_lang}:\n\n{text}"
    },
    "explain_like": {
        "system": "You are an expert teacher. Explain the concept at the level appropriate for the audience. Keep it to 2-3 sentences.",
        "user": "Explain '{concept}' as if I am {audience}."
    }
}


def use_template(template_name, **kwargs):
    """Build messages from a template in the library."""
    tmpl = TEMPLATES[template_name]
    return [
        {"role": "system", "content": tmpl["system"].format(**kwargs)},
        {"role": "user", "content": tmpl["user"].format(**kwargs)}
    ]


# --- Use the summarize template ---
print("=" * 60)
print("  TEMPLATE: summarize")
print("=" * 60)
messages = use_template(
    "summarize",
    length="one sentence",
    text="Artificial intelligence has transformed industries from healthcare to finance. Machine learning models can now diagnose diseases, predict market trends, and automate complex workflows that previously required human expertise."
)
show_messages(messages)
_ = chat(messages, temperature=0.3)

  TEMPLATE: summarize


AI has advanced to include diagnosing diseases, predicting market trends, and automating complex tasks previously requiring human expertise.


⏱️ 0.38s | 140 chars


In [44]:
# --- Use the translate template ---
print("=" * 60)
print("  TEMPLATE: translate")
print("=" * 60)
messages = use_template(
    "translate",
    source_lang="English",
    target_lang="French",
    text="The best way to predict the future is to create it."
)
show_messages(messages)
_ = chat(messages, temperature=0.3)

  TEMPLATE: translate


La meilleure façon de prévoir l'avenir est d'en créer.


⏱️ 0.28s | 54 chars


In [45]:
# --- Use the explain_like template ---
print("=" * 60)
print("  TEMPLATE: explain_like")
print("=" * 60)
messages = use_template(
    "explain_like",
    concept="blockchain",
    audience="a 12-year-old who loves video games"
)
show_messages(messages)
_ = chat(messages, temperature=0.5)

# Same template, different audience
print("\n" + "=" * 60)
print("  TEMPLATE: explain_like (different audience)")
print("=" * 60)
messages = use_template(
    "explain_like",
    concept="blockchain",
    audience="a bank executive evaluating new technology investments"
)
show_messages(messages)
_ = chat(messages, temperature=0.5)

  TEMPLATE: explain_like


Imagine blockchain is like a big, unbreakable online diary where all your game moves are recorded and shared with everyone in the world at once! It's super secure because even if someone tries to change what you did on the game, it would be like trying to erase their whole history from that diary. This way, no one can cheat or trick other players, just like how a video game is fair for all its users.


⏱️ 0.98s | 403 chars

  TEMPLATE: explain_like (different audience)


Blockchain, in banking terms, is like a shared ledger where every transaction is recorded and verified by multiple parties. It's decentralized, meaning no single entity controls it, which makes it highly secure and transparent. Think of it as a global register that keeps track of all financial transactions without needing a central authority to validate them. This technology can revolutionize how banks handle money transfers, reduce fraud, and improve the efficiency of cross-border payments by eliminating intermediaries like banks.


⏱️ 1.24s | 537 chars


### Experiment 5D: LangChain PromptTemplate

LangChain provides a built-in `ChatPromptTemplate` for more robust template management.

In [46]:
from langchain_core.prompts import ChatPromptTemplate

# Define a reusable LangChain prompt template
email_template = ChatPromptTemplate.from_messages([
    ("system", "You are a professional email writer. Write concise, polite emails. Tone: {tone}."),
    ("user", "Write an email to {recipient} about: {subject}\n\nKey points to include:\n{key_points}")
])

# Generate messages from the template
formatted = email_template.invoke({
    "tone": "formal but friendly",
    "recipient": "a client",
    "subject": "Project timeline update",
    "key_points": "- Phase 1 completed ahead of schedule\n- Phase 2 starts next Monday\n- Need feedback on the design mockups by Friday"
})

print("LANGCHAIN ChatPromptTemplate")
print("=" * 60)

# Show what was generated
for msg in formatted.messages:
    role = msg.__class__.__name__.replace("Message", "")
    print(f"\n[{role}]\n{msg.content}")

print(f"\n{'=' * 60}")
print("RESPONSE:")
print("=" * 60)

response = llm.invoke(formatted)
display(Markdown(response.content))

LANGCHAIN ChatPromptTemplate

[System]
You are a professional email writer. Write concise, polite emails. Tone: formal but friendly.

[Human]
Write an email to a client about: Project timeline update

Key points to include:
- Phase 1 completed ahead of schedule
- Phase 2 starts next Monday
- Need feedback on the design mockups by Friday

RESPONSE:


Subject: Update on Our Project Timeline

Dear [Client's Name],

I hope this message finds you well.

Thank you for your continued support and interest in our project. I am writing to provide you with an update on our progress, particularly regarding the timeline of our current phase.

Phase 1 has been completed ahead of schedule as we have made efficient use of the resources allocated to us. We are excited to move forward with Phase 2, which will commence next Monday.

Feedback is integral to ensuring that our work meets your expectations. Therefore, we kindly request a review of the design mockups by Friday at noon. This feedback is crucial for refining the final designs and addressing any concerns you might have.

Please let us know if there are any specific items or aspects of the designs that require more detailed consideration. We value your input greatly and appreciate your time in reviewing this important phase.

Should you need further clarification on the timeline, feel free to reach out at your convenience.

Thank you once again for your continued support and collaboration.

Best regards,

[Your Full Name]  
[Your Position]  
[Your Contact Information]

In [47]:
# Reuse the same template with totally different inputs
formatted2 = email_template.invoke({
    "tone": "casual and brief",
    "recipient": "a teammate",
    "subject": "Lunch plans",
    "key_points": "- Trying the new Thai place at noon\n- Need headcount by 11 AM\n- They have great vegetarian options"
})

print("SAME TEMPLATE, DIFFERENT INPUTS")
print("=" * 60)
response2 = llm.invoke(formatted2)
display(Markdown(response2.content))

print("\n💡 Same template produced a completely different email — that's the power of reusability!")

SAME TEMPLATE, DIFFERENT INPUTS


Subject: Lunch Plans - Thai Cuisine, Headcount Needed!

Hi [Teammate's Name],

Hope you're doing well! I'm excited about our lunch plans this week. We'll be treating ourselves to a taste of Thailand at noon today.

Here are the details:
- **Thai Place** (I haven't tried it yet but everyone seems excited)
- **Headcount Needed: 5**

Would appreciate your confirmation on who can join so we're not left without anyone.

Thanks for your help!

Let me know if you need anything else from my end. I'll be there to share in the Thai food paradise with a few friends and some laughs this afternoon.

Best,
[Your Name]


💡 Same template produced a completely different email — that's the power of reusability!


---

## 6. Sandbox — Try It Yourself!

Combine any techniques from this notebook.

In [48]:
# ============================================================
#  SANDBOX - Mix and match techniques!
# ============================================================

messages = [
    # Try: role-based system prompt
    {"role": "system", "content": "You are a helpful assistant. Respond in JSON format."},

    # Try: few-shot examples (uncomment to add)
    # {"role": "user", "content": "your example input"},
    # {"role": "assistant", "content": "your example output"},

    # Your actual prompt
    {"role": "user", "content": "What are 3 benefits of exercise?"}
]

print("YOUR CUSTOM EXPERIMENT")
print("=" * 60)
show_messages(messages)
_ = chat(messages, temperature=0.5)

YOUR CUSTOM EXPERIMENT


[
  "Improves cardiovascular health",
  "Enhances mood and mental well-being",
  "Increases muscle strength and endurance"
]


⏱️ 0.43s | 124 chars


---

## Key Takeaways

| Technique | When to Use | Reliability |
|-----------|-------------|-------------|
| **Zero-shot** | Simple, well-known tasks (translation, summarization) | Medium — format can vary |
| **One-shot** | When one example is enough to show the pattern | Good |
| **Few-shot** | Custom formats, domain-specific tasks, consistent output | High |
| **Role-based** | Controlling tone, expertise level, and perspective | High |
| **Structured output** | Pipelines, APIs, downstream processing | High with few-shot |
| **Templates** | Reusable tasks across many inputs | High — best for production |

### Combining Techniques — The Power Stack

```
┌─────────────────────────────────────────┐
│  SYSTEM: Role + format constraints      │  ← Role-based
├─────────────────────────────────────────┤
│  USER:   Example input 1                │
│  ASST:   Example output 1               │  ← Few-shot
│  USER:   Example input 2                │
│  ASST:   Example output 2               │
├─────────────────────────────────────────┤
│  USER:   Template.format(actual_data)   │  ← Template
├─────────────────────────────────────────┤
│  ASST:   Structured, consistent output  │  ← Reliable result
└─────────────────────────────────────────┘
```